In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
!ls ../../chandra/CSLU/data/agg-0/
!ls ../../chandra/CSLU/data/mod-agg-3/

AR  CA	FA  GE	HU  IT	KO  MY	PP  SD	SW  VI
BP  CZ	FR  HI	IN  JA	MA  PO	RU  SP	TA
AR  BP	CA  FA	FR  GE	HI  HU	IN  IT	JA  KO	MY  PO	PP  RU	SD  SP	SW  VI


In [29]:
from os import listdir

#dataroot = '/exp/data/CSLU-AccentedEnglish/cslu_fae/speech'
dataroot = '../../chandra/CSLU/data/mod-agg-3'

dirs = [f for f in listdir(dataroot)]
len(dirs), dirs

(20,
 ['IN',
  'PO',
  'AR',
  'PP',
  'KO',
  'HU',
  'SW',
  'SP',
  'FR',
  'SD',
  'MY',
  'JA',
  'VI',
  'IT',
  'BP',
  'FA',
  'HI',
  'CA',
  'RU',
  'GE'])

In [8]:
# s = input()
## AR  Arabic BP  Brazillian Portuguese CA  Cantonese CZ  Czech FA  Farsi FR  French GE  German HI  Hindi HU  Hungarian IN  Indonesian IT  Italian JA  Japanese KO  Korean MA  Mandarin MY  Malay PO  Polish PP  Iberian Portuguese RU  Russian SD  Swedish SP  Spanish SW  Swahili TA  Tamil VI  Vietnamese

AR  Arabic BP  Brazillian Portuguese CA  Cantonese CZ  Czech FA  Farsi FR  French GE  German HI  Hindi HU  Hungarian IN  Indonesian IT  Italian JA  Japanese KO  Korean MA  Mandarin MY  Malay PO  Polish PP  Iberian Portuguese RU  Russian SD  Swedish SP  Spanish SW  Swahili TA  Tamil VI  Vietnamese


In [9]:
from collections import defaultdict

st = s.split()
accent_mp = {}
for i in range(len(st)):
    if len(st[i]) == 2:
        accent_mp[st[i]] = ''
        key = st[i]
    else:
        accent_mp[key] = accent_mp[key] + f'{st[i]} '
print(accent_mp)

{'AR': 'Arabic ', 'BP': 'Brazillian Portuguese ', 'CA': 'Cantonese ', 'CZ': 'Czech ', 'FA': 'Farsi ', 'FR': 'French ', 'GE': 'German ', 'HI': 'Hindi ', 'HU': 'Hungarian ', 'IN': 'Indonesian ', 'IT': 'Italian ', 'JA': 'Japanese ', 'KO': 'Korean ', 'MA': 'Mandarin ', 'MY': 'Malay ', 'PO': 'Polish ', 'PP': 'Iberian Portuguese ', 'RU': 'Russian ', 'SD': 'Swedish ', 'SP': 'Spanish ', 'SW': 'Swahili ', 'TA': 'Tamil ', 'VI': 'Vietnamese '}


In [30]:
import os 

for _dir in dirs:
    print(f'{accent_mp[_dir]} \t {len(listdir(os.path.join(dataroot, _dir)))}')

Indonesian  	 277
Polish  	 443
Arabic  	 327
Iberian Portuguese  	 203
Korean  	 479
Hungarian  	 800
Swahili  	 180
Spanish  	 663
French  	 841
Swedish  	 637
Malay  	 156
Japanese  	 496
Vietnamese  	 255
Italian  	 635
Brazillian Portuguese  	 1558
Farsi  	 747
Hindi  	 1123
Cantonese  	 677
Russian  	 667
German  	 891


In [11]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os, sys, json, random
import librosa

In [86]:
from multiprocessing import Pool, Manager

def process_samples(_dir):
    print("loading", accent_mp[_dir])
    samples_lst = [item for item in os.listdir(os.path.join(dataroot, _dir))]
    for sample in tqdm(samples_lst):
        try:
            name = sample.replace('.wav', '')
            path = os.path.join(dataroot, _dir, sample)
            duration = librosa.get_duration(filename=path)
            if duration > 30:
                continue
            L.append({
                "name": name,
                "path": path,
                "accent": accent_mp[_dir]
            })
        except Exception as e:
            print(str(path), e)
            pass

all_accents = list(accent_mp.keys())
not_accents = ['CZ', 'MA', 'TA', 'FR', 'IT', 'SW', 'VI']
train_accents = list(set(all_accents) - set(not_accents))

manager = Manager()
L = manager.list()
pool = Pool(16)
pool.map(process_samples, train_accents)
pool.close()
pool.join()
print(len(L))

loadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloading          loading  Russian Brazillian Portuguese German Hindi loadingloadingMalay Swedish Polish Indonesian Spanish Farsi Arabic  Iberian Portuguese 



 



 


Hungarian 
Korean 
Cantonese 

loading Japanese 


100%|██████████| 1558/1558 [00:02<00:00, 744.80it/s]


10144


In [87]:
data = list(L)
print(len(data))
print(data[:3])

10144
[{'name': 'FAR00523-chunk-00', 'path': '../../chandra/CSLU/data/mod-agg-3/AR/FAR00523-chunk-00.wav', 'accent': 'Arabic '}, {'name': 'FIN00145-chunk-02', 'path': '../../chandra/CSLU/data/mod-agg-3/IN/FIN00145-chunk-02.wav', 'accent': 'Indonesian '}, {'name': 'FAR00576-chunk-00', 'path': '../../chandra/CSLU/data/mod-agg-3/AR/FAR00576-chunk-00.wav', 'accent': 'Arabic '}]


In [88]:
df = pd.DataFrame(data)
df.head()

,name,path,accent
0,FAR00523-chunk-00,../../chandra/CSLU/data/mod-agg-3/AR/FAR00523-...,Arabic
1,FIN00145-chunk-02,../../chandra/CSLU/data/mod-agg-3/IN/FIN00145-...,Indonesian
2,FAR00576-chunk-00,../../chandra/CSLU/data/mod-agg-3/AR/FAR00576-...,Arabic
3,FMY00139-chunk-01,../../chandra/CSLU/data/mod-agg-3/MY/FMY00139-...,Malay
4,FPP00035-chunk-00,../../chandra/CSLU/data/mod-agg-3/PP/FPP00035-...,Iberian Portuguese


In [62]:
# df['mp3path'] = df['path'].str.replace('.wav', '.mp3').replace('wav', 'clips')
# df.head()

In [89]:
## Filter broken and non-existed paths

print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

# df = df.sample(frac=1)
# df = df.reset_index(drop=True)
df.head()

Step 0: 10144
Step 1: 10144


,name,path,accent
0,FAR00523-chunk-00,../../chandra/CSLU/data/mod-agg-3/AR/FAR00523-...,Arabic
1,FIN00145-chunk-02,../../chandra/CSLU/data/mod-agg-3/IN/FIN00145-...,Indonesian
2,FAR00576-chunk-00,../../chandra/CSLU/data/mod-agg-3/AR/FAR00576-...,Arabic
3,FMY00139-chunk-01,../../chandra/CSLU/data/mod-agg-3/MY/FMY00139-...,Malay
4,FPP00035-chunk-00,../../chandra/CSLU/data/mod-agg-3/PP/FPP00035-...,Iberian Portuguese


In [90]:
print("Labels: ", df["accent"].unique())
print()
df.groupby("accent").count()[["path"]]

Labels:  ['Arabic ' 'Indonesian ' 'Malay ' 'Iberian Portuguese ' 'Polish '
 'Russian ' 'German ' 'Spanish ' 'Farsi ' 'Brazillian Portuguese '
 'Hindi ' 'Swedish ' 'Korean ' 'Cantonese ' 'Japanese ' 'Hungarian ']



,path
accent,
Arabic,327
Brazillian Portuguese,1558
Cantonese,677
Farsi,747
German,891
Hindi,1123
Hungarian,800
Iberian Portuguese,203
Indonesian,277


## grouping 4:
brazillian portuguese, iberian portuguese, spanish <br>
korean, japanese, mandarin, cantonese <br>
polish, czech, russian <br>
tamil, malay, indonesian <br>
arabic, farsi, hindi <br>
swedish, german, Hungarian <br>

In [48]:
groups = input()
groups = groups.split('<br>')
accent2group = {}
for i, group in enumerate(groups):
    if len(group):
        for accent in group.split(','):
            accent2group[accent.strip().lower()] = i

brazillian portuguese, iberian portuguese, spanish <br> korean, japanese, mandarin, cantonese <br> polish, czech, russian <br> tamil, malay, indonesian <br> arabic, farsi, hindi <br> swedish, german, Hungarian <br>


In [91]:
accent2group

{'brazillian portuguese': 0,
 'iberian portuguese': 0,
 'spanish': 0,
 'korean': 1,
 'japanese': 1,
 'mandarin': 1,
 'cantonese': 1,
 'polish': 2,
 'czech': 2,
 'russian': 2,
 'tamil': 3,
 'malay': 3,
 'indonesian': 3,
 'arabic': 4,
 'farsi': 4,
 'hindi': 4,
 'swedish': 5,
 'german': 5,
 'hungarian': 5}

In [92]:
df["accent"] = df["accent"].str.lower().str.strip()
df["accent"].replace(accent2group, inplace=True)
df.head()

,name,path,accent
0,FAR00523-chunk-00,../../chandra/CSLU/data/mod-agg-3/AR/FAR00523-...,4
1,FIN00145-chunk-02,../../chandra/CSLU/data/mod-agg-3/IN/FIN00145-...,3
2,FAR00576-chunk-00,../../chandra/CSLU/data/mod-agg-3/AR/FAR00576-...,4
3,FMY00139-chunk-01,../../chandra/CSLU/data/mod-agg-3/MY/FMY00139-...,3
4,FPP00035-chunk-00,../../chandra/CSLU/data/mod-agg-3/PP/FPP00035-...,0


In [93]:
print("Labels: ", df["accent"].unique())
print()
print(df.groupby("accent").count()[["path"]])

Labels:  [4 3 0 2 5 1]

        path
accent      
0       2424
1       1652
2       1110
3        433
4       2197
5       2328


In [94]:
!mkdir -p CSLU_training_data/grouping_4

In [95]:
save_path = "CSLU_training_data/grouping_4"

train_df, rem_df = train_test_split(df, train_size=0.8, random_state=101, stratify=df["accent"])
dev_df, test_df = train_test_split(rem_df, test_size=0.5, random_state=101, stratify=rem_df['accent'])

train_df = train_df.reset_index(drop=True)
dev_df = dev_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
dev_df.to_csv(f"{save_path}/dev.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)

print(train_df.shape)
print(dev_df.shape)
print(test_df.shape)

(8115, 3)
(1014, 3)
(1015, 3)


##
1. try on smaller audio based on silence --> ping Ma'am
2. try on simpler architecture
3.a use wav2vec2 feature architecute and then use those features for classification directly
3.b user 3 with RNN or LSTM architecture 
3.c maybe see TDNN
***4. maybe check whisper
5. check papers shares on chat

## EVAL

In [71]:
# need to save the model first
import librosa
import torch
import torchaudio
from sklearn.metrics import classification_report
from datasets import load_dataset, load_metric

from transformers import AutoConfig, Wav2Vec2Processor

In [72]:
test_dataset = load_dataset("csv", data_files={"test": "CSLU_training_data/init/test.csv"}, delimiter="\t")["test"]
test_dataset

Using custom data configuration default-7b74c9463294d8e2


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/mayur/.cache/huggingface/datasets/csv/default-7b74c9463294d8e2/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['name', 'path', 'accent'],
    num_rows: 474
})

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [83]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, Tuple
import torch
import torch.nn as nn
import transformers
from transformers import Wav2Vec2Processor
from transformers.file_utils import ModelOutput

@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None
    h1: Optional[Tuple[torch.FloatTensor]] = None
    h2: Optional[Tuple[torch.FloatTensor]] = None
        
class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense1 = nn.Linear(config.hidden_size, 300)
        self.dense2 = nn.Linear(300, 100)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(100, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense1(x)
        x1 = torch.tanh(x)
        x2 = self.dropout(x1)
        x2 = self.dense2(x2)
        x2 = torch.tanh(x2)
        x3 = self.dropout(x2)        
        x3 = self.out_proj(x3)
        return x1, x2, x3

class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()
        for module in self.wav2vec2.encoder.layers[:10]:
            for param in module.parameters():
                param.requires_grad = False

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=True,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

#        pdb.set_trace()

        hidden_states = outputs[0]
        #hidden_states = outputs[2][7]  ## taking output of 7th layer
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        h1, h2, logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (h1 + h2 + logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            h1=h1,
            h2=h2
        )

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [84]:
model_name_or_path = "CSLU_training_data/checkpoints/checkpoint-6500/"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [85]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)
    batch["speech"] = speech_array
    return batch

def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)
    input_values = features.input_values.to(device)
    attention_mask = None

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits 

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [86]:
import numpy as np
test_dataset = test_dataset.map(speech_file_to_array_fn)

  0%|          | 0/474 [00:00<?, ?ex/s]

/home/mayur/exp/anaconda3/envs/w2v2_new/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=8000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.
/home/mayur/exp/anaconda3/envs/w2v2_new/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=8000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.


In [87]:
result = test_dataset.map(predict, batched=True, batch_size=8)

Parameter 'function'=<function predict at 0x7ff17a1d42f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/60 [00:00<?, ?ba/s]

In [88]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

['Arabic ',
 'Brazillian Portuguese ',
 'Cantonese ',
 'Czech ',
 'Farsi ',
 'French ',
 'German ',
 'Hindi ',
 'Hungarian ',
 'Indonesian ',
 'Italian ',
 'Japanese ',
 'Korean ',
 'Mandarin ',
 'Polish ',
 'Russian ',
 'Spanish ',
 'Swedish ',
 'Tamil ',
 'Vietnamese ']

In [89]:
result

Dataset({
    features: ['name', 'path', 'accent', 'speech', 'predicted'],
    num_rows: 474
})

In [90]:
y_true = [config.label2id[name] for name in result["accent"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

[5, 7, 12, 0, 1]
[8, 7, 9, 0, 5]


In [91]:
print(classification_report(y_true, y_pred, target_names=label_names))

                        precision    recall  f1-score   support

               Arabic        0.17      0.36      0.23        11
Brazillian Portuguese        0.59      0.59      0.59        46
            Cantonese        0.67      0.23      0.34        26
                Czech        0.14      0.10      0.12        10
                Farsi        0.48      0.38      0.43        26
               French        0.26      0.28      0.27        29
               German        0.69      0.27      0.39        33
                Hindi        0.84      0.60      0.70        35
            Hungarian        0.34      0.64      0.44        28
           Indonesian        0.40      0.22      0.29         9
              Italian        0.19      0.27      0.23        22
             Japanese        0.37      0.37      0.37        19
               Korean        0.24      0.47      0.31        17
             Mandarin        0.42      0.57      0.48        28
               Polish        0.15      